# Описание проекта

Работа с базой данных, которая хранит информацию о венчурных фондах и инвестициях в компании-стартапы. Эта база данных основана на датасете Startup Investments, опубликованном на популярной платформе для соревнований по исследованию данных Kaggle.

ER-диаграмма
![ER-диаграмма](https://pictures.s3.yandex.net/resources/13_border2880_1628164784.png)

В проекте вам нужно проанализировать данные о фондах и инвестициях и написать запросы к базе.

### Посчитайте, сколько компаний закрылось.

SELECT COUNT(status)  
FROM company  
WHERE status = 'closed'

### Отобразите количество привлечённых средств для новостных компаний США. 
Используйте данные из таблицы `company`. Отсортируйте таблицу по убыванию значений в поле `funding_total` .

SELECT funding_total  
FROM company  
WHERE category_code = 'news' AND country_code = 'USA'  
ORDER BY funding_total DESC

### Найдите общую сумму сделок по покупке одних компаний другими в долларах. 
Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

SELECT SUM(price_amount)  
FROM acquisition  
WHERE EXTRACT(year FROM acquired_at) BETWEEN 2011 AND 2013 AND term_code = 'cash'

### Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.

SELECT first_name, last_name, twitter_username  
FROM people  
WHERE twitter_username LIKE 'Silver%'

### Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'.

SELECT *  
FROM people  
WHERE twitter_username LIKE '%money%' AND last_name LIKE 'K%'

### Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. 
Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

SELECT country_code, SUM(funding_total)  
FROM company  
GROUP BY country_code  
ORDER BY SUM(funding_total) DESC

### Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.  
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

SELECT funded_at,  
       MIN(raised_amount),  
       MAX(raised_amount)  
FROM funding_round  
GROUP BY funded_at  
HAVING MIN(raised_amount) <> 0 AND MIN(raised_amount) <> MAX(raised_amount)

### Создайте поле с категориями:  
- Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.  
- Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.  
Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.  
Отобразите все поля таблицы fund и новое поле с категориями.

SELECT *,  
        CASE  
            WHEN invested_companies >= 100 THEN 'high_activity'  
            WHEN invested_companies >= 20 AND invested_companies < 100 THEN 'middle_activity'  
            ELSE 'low_activity'  
        END  
FROM fund

### Для каждой из категорий, назначенных в предыдущем задании, 
посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

SELECT ROUND(AVG(investment_rounds)) AS avg_,  
       CASE  
           WHEN invested_companies>=100 THEN 'high_activity'  
           WHEN invested_companies>=20 THEN 'middle_activity'  
           ELSE 'low_activity'  
       END AS activity  
FROM fund  
GROUP BY activity  
ORDER BY avg_

### Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы.  
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. Выгрузите десять самых активных стран-инвесторов.  
Отсортируйте таблицу по среднему количеству компаний от большего к меньшему, а затем по коду страны в лексикографическом порядке.

SELECT country_code,  
        MIN(invested_companies) AS min_,  
        MAX(invested_companies) AS max_,  
        AVG(invested_companies) AS avg_  
FROM fund  
WHERE EXTRACT(year FROM founded_at) BETWEEN 2010 AND 2012  
GROUP BY country_code  
HAVING MIN(invested_companies) <> 0  
ORDER BY avg_ DESC, country_code  
LIMIT 10

### Отобразите имя и фамилию всех сотрудников стартапов. 
Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

SELECT p.first_name,  
        p.last_name,  
        e.instituition  
FROM people AS p  
LEFT JOIN education AS e ON p.id = e.person_id

### Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники.
Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

SELECT c.name,  
        COUNT(DISTINCT e.instituition) AS count_  
FROM company AS c  
JOIN people AS p ON c.id = p.company_id  
JOIN education AS e ON p.id = e.person_id  
GROUP BY c.name  
ORDER BY count_ DESC  
LIMIT 5

### Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

WITH  
f AS (SELECT company_id  
    FROM funding_round  
    WHERE is_first_round + is_last_round = 2),  
c AS (SELECT name, id  
      FROM company AS c  
      WHERE status = 'closed')  
SELECT DISTINCT c.name  
FROM c  
INNER JOIN f ON c.id = f.company_id

### Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

SELECT id  
FROM people  
WHERE company_id IN  
            (SELECT id  
            FROM company  
            WHERE status = 'closed' AND id IN (SELECT company_id  
                                              FROM funding_round  
                                              WHERE is_first_round+is_last_round = 2))

### Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

SELECT DISTINCT e.person_id, e.instituition  
          FROM education AS e  
          WHERE e.person_id IN (  
                        SELECT DISTINCT p.id  
                        FROM people AS p  
                        WHERE company_id IN (  
                                SELECT DISTINCT id  
                                FROM company  
                                WHERE id IN (  
                                             SELECT company_id  
                                             FROM funding_round  
                                             WHERE is_first_round = 1 AND is_last_round = 1)  
                                             AND status = 'closed'))

### Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. 
При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

SELECT DISTINCT e.person_id, COUNT(e.instituition)  
          FROM education AS e  
          WHERE e.person_id IN (  
                        SELECT DISTINCT p.id  
                        FROM people AS p  
                        WHERE company_id IN (  
                                SELECT DISTINCT id  
                                FROM company  
                                WHERE id IN (  
                                             SELECT company_id  
                                             FROM funding_round  
                                             WHERE is_first_round = 1 AND is_last_round = 1)  
                                             AND status = 'closed'))  
GROUP BY e.person_id

### Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. 
Нужно вывести только одну запись, группировка здесь не понадобится.

SELECT AVG(tab.count)  
FROM  
(SELECT DISTINCT e.person_id, COUNT(e.instituition)  
          FROM education AS e  
          WHERE e.person_id IN (  
                        SELECT DISTINCT p.id  
                        FROM people AS p  
                        WHERE company_id IN (  
                                SELECT DISTINCT id  
                                FROM company  
                                WHERE id IN (  
                                             SELECT company_id  
                                             FROM funding_round  
                                             WHERE is_first_round = 1 AND is_last_round = 1)  
                                             AND status = 'closed'))  
GROUP BY e.person_id) AS tab

### Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook.

SELECT AVG(count)  
FROM  
    (SELECT person_id, COUNT(instituition)  
    FROM education  
    WHERE person_id IN  
        (SELECT id  
        FROM people  
        WHERE company_id IN  
            (SELECT id  
            FROM company  
            WHERE name = 'Facebook'))  
    GROUP BY person_id) AS tab

### Составьте таблицу из полей:  
- name_of_fund — название фонда;  
- name_of_company — название компании;  
- amount — сумма инвестиций, которую привлекла компания в раунде.  
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

SELECT f.name AS name_of_fund,  
       c.name AS name_of_company,  
       f_r.raised_amount AS amount  
FROM investment AS i  
LEFT JOIN company AS c ON c.id = i.company_id  
LEFT JOIN fund AS f ON i.fund_id = f.id  
INNER JOIN   
            (SELECT*  
            FROM funding_round  
            WHERE funded_at BETWEEN '2012-01-01' AND '2013-12-31') AS f_r   
    ON f_r.id = i.funding_round_id  
WHERE c.milestones > 6

### Выгрузите таблицу, в которой будут такие поля:  
- название компании-покупателя;  
- сумма сделки;  
- название компании, которую купили;  
- сумма инвестиций, вложенных в купленную компанию;  
- доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.  
Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы.  
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

WITH   
an AS (SELECT c.name AS name_of_baer,  
                     a.price_amount AS transaction_amount,  
                     a.id AS id  
              FROM acquisition AS a  
              LEFT JOIN company AS c ON a.acquiring_company_id = c.id  
              WHERE a.price_amount > 0),  
ad AS (SELECT c.name AS business,  
                    c.funding_total AS investment,  
                    a.id AS id  
             FROM acquisition AS a  
             LEFT JOIN company AS c ON a.acquired_company_id = c.id  
             WHERE c.funding_total > 0)  
SELECT a_n.name_of_baer,  
       a_n.transaction_amount,  
       a_d.business,  
       a_d.investment,  
       ROUND(a_n.transaction_amount / a_d.investment) AS fraction  
FROM an AS a_n  
JOIN ad AS a_d ON a_n.id = a_d.id  
ORDER BY transaction_amount DESC, business  
LIMIT 10

### Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. 
Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

SELECT c.name,  
        EXTRACT(month FROM f_r.funded_at)  
FROM company AS c  
LEFT JOIN funding_round AS f_r ON c.id = f_r.company_id  
WHERE c.category_code = 'social'  
        AND EXTRACT(year FROM funded_at) BETWEEN 2010 AND 2013 AND raised_amount <> 0

### Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
- номер месяца, в котором проходили раунды;  
- количество уникальных названий фондов из США, которые инвестировали в этом месяце;  
- количество компаний, купленных за этот месяц;  
- общая сумма сделок по покупкам в этом месяце.

WITH  
tab_1 AS  
    (SELECT EXTRACT(month FROM fr.funded_at) AS f_month,  
    COUNT(DISTINCT f.id) AS count_funds  
    FROM fund AS f  
    LEFT JOIN investment AS i ON f.id = i.fund_id  
    LEFT JOIN funding_round AS fr ON i.funding_round_id = fr.id  
    WHERE f.country_code = 'USA'  
    AND EXTRACT(YEAR FROM CAST(fr.funded_at AS DATE)) BETWEEN 2010 AND 2013  
    GROUP BY f_month),  
tab_2 AS  
    (SELECT EXTRACT(MONTH FROM acquired_at) AS f_month,  
    COUNT(acquired_company_id) AS count_company,  
    SUM(price_amount) AS sum_total  
    FROM acquisition  
    WHERE EXTRACT(YEAR FROM CAST(acquired_at AS DATE)) BETWEEN 2010 AND 2013  
    GROUP BY f_month)  
SELECT f.f_month, f.count_funds, a.count_company, a.sum_total  
FROM tab_1 AS f  
LEFT JOIN tab_2 AS a ON f.f_month = a.f_month

### Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. 
Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

WITH  
year_2011 AS  
    (SELECT country_code,  
    AVG(funding_total) AS avg_2011  
    FROM company  
    WHERE EXTRACT(year FROM founded_at) = 2011  
    GROUP BY country_code),  

year_2012 AS  
    (SELECT country_code,  
    AVG(funding_total) AS avg_2012  
    FROM company  
    WHERE EXTRACT(year FROM founded_at) = 2012  
    GROUP BY country_code),  

year_2013 AS  
    (SELECT country_code,  
    AVG(funding_total) AS avg_2013  
    FROM company  
    WHERE EXTRACT(year FROM founded_at) = 2013  
    GROUP BY country_code)  

    
SELECT year_2011.country_code,  
         year_2011.avg_2011,  
         year_2012.avg_2012,  
         year_2013.avg_2013  
FROM year_2011  
INNER JOIN year_2012 ON year_2011.country_code = year_2012.country_code  
INNER JOIN year_2013 ON year_2012.country_code = year_2013.country_code  
ORDER BY year_2011.avg_2011 DESC  